# COGS 108 - Final Project 

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that PIDs will be scraped from the public submission, but student names will be included.)

* [  ] YES - make available
* [  ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Michael Baluja
- Griffin Mittleman
- Deepkiran Sangha
- Hannah Williams
- Raymart Evangelista

# Group Members IDs

- A14499660
- A15734955
- A15096006
- A16112910
- A16094994

# Research Question

How is the further integration of job automation into the United States’ workforce related to employment salaries and rates across various job industries in the past 25 years?

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


We hypothesize that occupations with higher salaries will have a lower probability of being automated, and that these jobs with high automation probabilities will show a decrease in employment rates for a period of time surrounding the point at which the automation likelihood was estimated.

# Dataset(s)

Outline: (Delete before final submission!)
*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

- Dataset Name: Occupations by State and Likelihood of Automation
- Link to the dataset: https://data.world/wnedds/occupations-by-state-and-likelihood-of-automation
- Number of observations: 702 (1 per occupation)

This dataset includes job title, OCC code (used to categorize), probability of occupation, and number of employees per state



- Dataset Name: Wage by Occupation
- Link to the dataset: https://data.world/quanticdata/occupation-and-salary-by-state-and-likelihood-of-automation/workspace/file?filename=national_M2016_dl.xlsx
- Number of observations: 1394

This dataset includes job titles, OCC code, and various wage metrics per occupation (salary vs hourly, average pay, statistics for these fields, North American Industry Classification System code, employee number data, etc)

The two datasets providing information by occupations will be combined by their OCC code, since it is the most standard metric they share (note: some datasets have different occupation name string formatting, so this is not as easily used). Since we mostly want to consider this data in terms of probability of automation, data from the second dataset will only be kept if there is a corresponding OCC code in the first dataset.

# Setup

In [3]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [4]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*

# COGS 108 - Final Project 